In [442]:
import pandas as pd 
import numpy as np 
from tqdm import tqdm
import math
import gc

In [443]:
# ! cp data/Train_target.csv ./raw_data/Train_target.csv
# ! cp data/Test_target.csv ./raw_data/Test_target.csv

In [444]:
train=pd.read_csv("Train.csv")
submission=pd.read_csv("Test.csv")
airqo=pd.read_csv("airqo_metadata.csv")

In [445]:
train=train.merge(airqo, on='location', how='left')
submission=submission.merge(airqo, on='location', how='left')
train.head(1)

,ID,location,temp,precip,rel_humidity,wind_dir,wind_spd,atmos_press,target,loc_altitude,km2,aspect,dist_tertiary,dist_unclassified,dist_residential,popn,hh,hh_cook_charcoal,hh_cook_firewood,hh_burn_waste
0,ID_train_0,C,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...",45.126304,1178.3,8.5,168.7,72.45934,406.967815,13.93135,97895,26873,21316,751,9835


In [446]:
# functions to convert features  fron string to List of values    
features=["temp","precip","rel_humidity","wind_dir","wind_spd","atmos_press"]

def replace_nan(x):
    if x==" ":
        return np.nan
    else :
        return float(x)
    
for feature in features : 
    train[feature]=train[feature].apply(lambda x: [ replace_nan(X) for X in x.replace("nan"," ").split(",")])
    submission[feature]=submission[feature].apply(lambda x: [ replace_nan(X)  for X in x.replace("nan"," ").split(",")])

In [447]:
train.head(1)

,ID,location,temp,precip,rel_humidity,wind_dir,wind_spd,atmos_press,target,loc_altitude,km2,aspect,dist_tertiary,dist_unclassified,dist_residential,popn,hh,hh_cook_charcoal,hh_cook_firewood,hh_burn_waste
0,ID_train_0,C,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",45.126304,1178.3,8.5,168.7,72.45934,406.967815,13.93135,97895,26873,21316,751,9835


In [448]:
submission.head(1)

,ID,location,temp,precip,rel_humidity,wind_dir,wind_spd,atmos_press,loc_altitude,km2,aspect,dist_tertiary,dist_unclassified,dist_residential,popn,hh,hh_cook_charcoal,hh_cook_firewood,hh_burn_waste
0,ID_test_0,C,"[23.96666667, 22.88333333, 22.28333333, 21.483...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.6898333329999999, 0.722333333, 0.7755833329...","[290.1352379, 328.47011289999995, 323.5730796,...","[0.6333333329999999, 0.725, 0.955, 0.8725, 0.6...","[87.64333333, 87.67, 87.70583333, 87.75083333,...",1178.3,8.5,168.7,72.45934,406.967815,13.93135,97895,26873,21316,751,9835


In [449]:
submission.shape

(5035, 19)

In [450]:
# clean train dataframe
def aggregate_features(x,col_name):
    x["max_"+col_name]=x[col_name].apply(np.max)
    x["min_"+col_name]=x[col_name].apply(np.min)
    x["mean_"+col_name]=x[col_name].apply(np.mean)
    x["std_"+col_name]=x[col_name].apply(np.std)
    x["ptp_"+col_name]=x[col_name].apply(np.ptp)
    return x  
   
for col_name in tqdm(features):
    train=aggregate_features(train,col_name)
    
train.drop(features,1,inplace=True)
train = train.dropna()
train.head(1)

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:13<00:00,  2.29s/it]


,ID,location,target,loc_altitude,km2,aspect,dist_tertiary,dist_unclassified,dist_residential,popn,...,max_wind_spd,min_wind_spd,mean_wind_spd,std_wind_spd,ptp_wind_spd,max_atmos_press,min_atmos_press,mean_atmos_press,std_atmos_press,ptp_atmos_press
1,ID_train_1,D,79.131702,980.8,0.8,90.0,339.693908,76.024389,11.601442,3596,...,3.009167,0.248333,0.899208,0.539199,2.760833,90.725,90.056667,90.429924,0.156,0.668333


In [451]:
train.shape

(7212, 44)

In [452]:
# clean test dataframe 
for col_name in tqdm(features):
    submission=aggregate_features(submission,col_name)
    
submission.drop(features,1,inplace=True)
submission.head(1)

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:04<00:00,  1.37it/s]


,ID,location,loc_altitude,km2,aspect,dist_tertiary,dist_unclassified,dist_residential,popn,hh,...,max_wind_spd,min_wind_spd,mean_wind_spd,std_wind_spd,ptp_wind_spd,max_atmos_press,min_atmos_press,mean_atmos_press,std_atmos_press,ptp_atmos_press
0,ID_test_0,C,1178.3,8.5,168.7,72.45934,406.967815,13.93135,97895,26873,...,1.915833,0.171667,0.812555,0.392783,1.744167,88.020833,87.474167,87.748506,0.132764,0.546667


In [453]:
submission.shape

(5035, 43)

In [454]:
# train, test split for the TRAIN DATA
cols = ["target", "ID"]
X = pd.get_dummies(train.drop(cols, axis=1), columns=["location"], prefix=["location_is"]) 
y = train['target']
X.head(1)

,loc_altitude,km2,aspect,dist_tertiary,dist_unclassified,dist_residential,popn,hh,hh_cook_charcoal,hh_cook_firewood,...,max_atmos_press,min_atmos_press,mean_atmos_press,std_atmos_press,ptp_atmos_press,location_is_A,location_is_B,location_is_C,location_is_D,location_is_E
1,980.8,0.8,90.0,339.693908,76.024389,11.601442,3596,663,7,651,...,90.725,90.056667,90.429924,0.156,0.668333,0,0,0,1,0


In [455]:
X.shape

(7212, 46)

In [456]:
y.shape

(7212,)

In [457]:
submission.head(1)

,ID,location,loc_altitude,km2,aspect,dist_tertiary,dist_unclassified,dist_residential,popn,hh,...,max_wind_spd,min_wind_spd,mean_wind_spd,std_wind_spd,ptp_wind_spd,max_atmos_press,min_atmos_press,mean_atmos_press,std_atmos_press,ptp_atmos_press
0,ID_test_0,C,1178.3,8.5,168.7,72.45934,406.967815,13.93135,97895,26873,...,1.915833,0.171667,0.812555,0.392783,1.744167,88.020833,87.474167,87.748506,0.132764,0.546667


In [458]:
# Submission DATA
droppers = ['ID']
x_submission = pd.get_dummies(submission.drop(droppers, axis=1), columns=["location"], prefix=["location_is"])
x_submission.head(2)

,loc_altitude,km2,aspect,dist_tertiary,dist_unclassified,dist_residential,popn,hh,hh_cook_charcoal,hh_cook_firewood,...,max_atmos_press,min_atmos_press,mean_atmos_press,std_atmos_press,ptp_atmos_press,location_is_A,location_is_B,location_is_C,location_is_D,location_is_E
0,1178.3,8.5,168.7,72.45934,406.967815,13.93135,97895,26873,21316,751,...,88.020833,87.474167,87.748506,0.132764,0.546667,0,0,1,0,0
1,1178.3,8.5,168.7,72.45934,406.967815,13.93135,97895,26873,21316,751,...,88.181667,87.572500,87.936963,0.113756,0.609167,0,0,1,0,0


In [459]:
X.shape

(7212, 46)

In [460]:
# Scale TRAIN data
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

X = pd.DataFrame(scale.fit_transform(X))
X.tail(1)

,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,40,41,42,43,44,45
7211,0.963759,1.888558,0.768273,-1.282756,0.951232,-0.37624,2.000137,2.00057,2.010701,0.763885,...,-1.114777,-1.081364,-1.127433,-1.40763,-1.030957,-0.424006,-0.239746,2.012566,-0.83268,-0.477705


In [461]:
X.shape

(7212, 46)

In [462]:
# Scale TEST data
x_submission = pd.DataFrame(scale.fit_transform(x_submission))
x_submission.tail(1)

,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,40,41,42,43,44,45
5034,0.873718,1.720574,0.253144,-1.392305,0.938359,-0.608394,1.904898,1.905315,1.9261,0.687372,...,-0.85321,-0.879546,-0.869906,-0.49173,-0.097458,-0.650786,-0.369899,1.927283,-0.585683,-0.359764


In [409]:
x_submission.shape

(5035, 46)

In [412]:
# train/test split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = .20)

In [413]:
# Simple Tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import ExtraTreeRegressor

# Multioutput
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import MultiTaskElasticNetCV

# Gaussian Process
from sklearn.gaussian_process import GaussianProcessRegressor

# Linear Regression, CV is for cross validation
from sklearn.linear_model.stochastic_gradient import SGDRegressor
from sklearn.linear_model import TheilSenRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model.passive_aggressive import PassiveAggressiveRegressor
from sklearn.linear_model import ARDRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import LarsCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LassoLarsCV

# Ensemble 
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble.weight_boosting import AdaBoostRegressor
from sklearn.ensemble.gradient_boosting import GradientBoostingRegressor
from sklearn.ensemble.bagging import BaggingRegressor
from sklearn.ensemble.forest import ExtraTreesRegressor
from sklearn.ensemble.forest import RandomForestRegressor

# Neighbours
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import RadiusNeighborsRegressor

# Neural Net
from sklearn.neural_network import MLPRegressor

# SVM
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
from sklearn.svm import NuSVR

# Also use Keras/Tensorflow for Deep Learning

In [414]:
regressors = [BaggingRegressor(n_estimators=50), ExtraTreesRegressor(n_estimators=50), RandomForestRegressor(n_estimators=50)]

# poor score for 
# PassiveAggressiveRegressor(), GaussianProcessRegressor(), SGDRegressor(), LassoLarsCV(), RANSACRegressor(),
# TheilSenRegressor(), AdaBoostRegressor(), LarsCV(), LassoCV(), Ridge(), BayesianRidge(), ElasticNetCV(), 
# LinearRegression(), HuberRegressor(), GradientBoostingRegressor(), DecisionTreeRegressor(), ExtraTreeRegressor(),
# KNeighborsRegressor(), RadiusNeighborsRegressor(weights='uniform'), MLPRegressor(), SVR(), LinearSVR(), NuSVR()

In [415]:
# Train each model
from sklearn.metrics import mean_absolute_error

for reg in regressors:
    reg.fit(x_train, y_train)
    y_pred = reg.predict(x_test)
    score = mean_absolute_error(y_pred, y_test)
    score = round(score, 5)
    print("The current model specifications are: ")
    print(reg)
    print("The mean absolute error for this model is: "+ str(score))
    print("  ")
    print("  ")

The current model specifications are: 
BaggingRegressor(base_estimator=None, bootstrap=True, bootstrap_features=False,
                 max_features=1.0, max_samples=1.0, n_estimators=50,
                 n_jobs=None, oob_score=False, random_state=None, verbose=0,
                 warm_start=False)
The mean absolute error for this model is: 16.43833
  
  
The current model specifications are: 
ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=50, n_jobs=None, oob_score=False,
                    random_state=None, verbose=0, warm_start=False)
The mean absolute error for this model is: 14.89971
  
  
The current model specifications are: 
RandomForestRegressor(b

In [327]:
# using the best model above
tree = ExtraTreesRegressor()

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

n_estimators = [450, 475, 490, 515, 520, 535, 550]
max_features = ['auto']
max_depth = [None]
min_samples_split = [2]
min_samples_leaf = [1]
bootstrap = [False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

{'n_estimators': [450, 475, 490, 515, 520, 535, 550], 'max_features': ['auto'], 'max_depth': [None], 'min_samples_split': [2], 'min_samples_leaf': [1], 'bootstrap': [False]}


In [417]:
# RandomizedSearchCV 
#tree_all_searched = RandomizedSearchCV(estimator = tree, param_distributions = random_grid, verbose=2, n_jobs = -1)
#tree_all = ExtraTreesRegressor(n_estimators = 562, max_features = 'auto', max_depth = 80, 
#                               min_samples_split = 2, min_samples_leaf = 1, bootstrap = False)
#tree_all_searched.fit(x_train, y_train)
#print("Done")

In [329]:
#tree_all_searched.best_params_

{'n_estimators': 535,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': None,
 'bootstrap': False}

In [418]:
#y_pred = tree_all_searched.predict(x_test)
#score = mean_absolute_error(y_pred, y_test)
#score = round(score, 5)
#print("This optimal model mean absolute error for this model is: "+ str(score))

In [465]:
# fit a optimal model to all of X, y
tree = ExtraTreesRegressor(n_estimators=50)
tree.fit(X, y)

ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=50, n_jobs=None, oob_score=False,
                    random_state=None, verbose=0, warm_start=False)

In [466]:
# Create Submission
x_submission.head(1)

,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,40,41,42,43,44,45
0,0.873718,1.720574,0.253144,-1.392305,0.938359,-0.608394,1.904898,1.905315,1.9261,0.687372,...,-1.056304,-1.004277,-1.031425,-0.793841,-1.120381,-0.650786,-0.369899,1.927283,-0.585683,-0.359764


In [467]:
# get rid of nan's
from sklearn.impute import SimpleImputer

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(x_submission)

x_submission = pd.DataFrame(imp.transform(x_submission))
x_submission.head(1)

,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,40,41,42,43,44,45
0,0.873718,1.720574,0.253144,-1.392305,0.938359,-0.608394,1.904898,1.905315,1.9261,0.687372,...,-1.056304,-1.004277,-1.031425,-0.793841,-1.120381,-0.650786,-0.369899,1.927283,-0.585683,-0.359764


In [468]:
# predict this submission using our Forest Model
x_submission_pred = tree.predict(x_submission)
x_submission['target'] = x_submission_pred

submission_df = x_submission.reset_index(drop=True)
submission_df.head(1)

,0,1,2,3,4,5,6,7,8,9,...,37,38,39,40,41,42,43,44,45,target
0,0.873718,1.720574,0.253144,-1.392305,0.938359,-0.608394,1.904898,1.905315,1.9261,0.687372,...,-1.004277,-1.031425,-0.793841,-1.120381,-0.650786,-0.369899,1.927283,-0.585683,-0.359764,85.052499


In [469]:
submission_df.shape

(5035, 47)

In [470]:
test_ids = pd.DataFrame(submission['ID']).reset_index()
test_ids.head(1)

,index,ID
0,0,ID_test_0


In [471]:
submission_df['ID'] = test_ids['ID']
submission_df.head(2)

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,target,ID
0,0.873718,1.720574,0.253144,-1.392305,0.938359,-0.608394,1.904898,1.905315,1.9261,0.687372,...,-1.031425,-0.793841,-1.120381,-0.650786,-0.369899,1.927283,-0.585683,-0.359764,85.052499,ID_test_0
1,0.873718,1.720574,0.253144,-1.392305,0.938359,-0.608394,1.904898,1.905315,1.9261,0.687372,...,-0.856480,-1.878170,-0.416535,-0.650786,-0.369899,1.927283,-0.585683,-0.359764,108.365579,ID_test_1


In [472]:
submission = submission_df[['ID', 'target']].round(2)
submission.head(1)

,ID,target
0,ID_test_0,85.05


In [473]:
submission.shape

(5035, 2)

In [475]:
submission.to_csv('submission_1.csv')

In [ ]:
## NEURAL NETWORK SECTION

In [55]:
# Using Keras Neural Network
import tensorflow as tf

In [121]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential

from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

In [123]:
# turn pandas to numpy array
x_train_net = x_train.values
y_train_net = y_train.values
x_test_net = x_test.values
y_test_net = y_test.values

In [124]:
x_train_net.shape

(5409, 17)

In [125]:
# model build
model = Sequential()
model.add(Dense(17, input_dim=17, kernel_initializer='normal', activation='relu'))
model.add(Dense(32, kernel_initializer='normal', activation='relu'))
model.add(Dense(64, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, kernel_initializer='normal', activation='relu'))
model.add(Dense(126, kernel_initializer='normal', activation='relu'))
model.add(Dense(256, kernel_initializer='normal', activation='relu'))
model.add(Dense(512, kernel_initializer='normal', activation='relu'))
model.add(Dense(256, kernel_initializer='normal', activation='relu'))
model.add(Dense(126, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64, kernel_initializer='normal', activation='relu'))
model.add(Dense(32, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, activation='relu'))

model.compile(optimizer='adam', loss = 'mean_squared_error', metrics=['mse'])

In [127]:
model.fit(x_train_net, y_train_net, epochs=100, batch_size=50, verbose=1)

Train on 5409 samples
Epoch 1/100
5409/5409 [==============================] - ETA: 2s - loss: 539.2827 - mean_squared_error: 539.28 - ETA: 1s - loss: 1066.6382 - mean_squared_error: 1066.63 - ETA: 1s - loss: 1232.7203 - mean_squared_error: 1232.72 - ETA: 1s - loss: 1175.0635 - mean_squared_error: 1175.06 - ETA: 1s - loss: 1235.6651 - mean_squared_error: 1235.66 - ETA: 0s - loss: 1327.7500 - mean_squared_error: 1327.75 - ETA: 0s - loss: 1346.0927 - mean_squared_error: 1346.09 - ETA: 0s - loss: 1348.6581 - mean_squared_error: 1348.65 - ETA: 0s - loss: 1346.1770 - mean_squared_error: 1346.17 - ETA: 0s - loss: 1316.1083 - mean_squared_error: 1316.10 - ETA: 0s - loss: 1314.3861 - mean_squared_error: 1314.38 - ETA: 0s - loss: 1341.6508 - mean_squared_error: 1341.65 - ETA: 0s - loss: 1373.6824 - mean_squared_error: 1373.68 - ETA: 0s - loss: 1364.9474 - mean_squared_error: 1364.94 - ETA: 0s - loss: 1363.3837 - mean_squared_error: 1363.38 - ETA: 0s - loss: 1391.1560 - mean_squared_error: 1391.

5409/5409 [==============================] - ETA: 0s - loss: 522.5779 - mean_squared_error: 522.57 - ETA: 1s - loss: 1339.6073 - mean_squared_error: 1339.60 - ETA: 1s - loss: 1529.4688 - mean_squared_error: 1529.46 - ETA: 1s - loss: 1371.4376 - mean_squared_error: 1371.43 - ETA: 1s - loss: 1492.3503 - mean_squared_error: 1492.35 - ETA: 1s - loss: 1371.0986 - mean_squared_error: 1371.09 - ETA: 1s - loss: 1319.6486 - mean_squared_error: 1319.64 - ETA: 0s - loss: 1374.9948 - mean_squared_error: 1374.99 - ETA: 0s - loss: 1330.8056 - mean_squared_error: 1330.80 - ETA: 0s - loss: 1288.7434 - mean_squared_error: 1288.74 - ETA: 0s - loss: 1290.9106 - mean_squared_error: 1290.91 - ETA: 0s - loss: 1297.9576 - mean_squared_error: 1297.95 - ETA: 0s - loss: 1322.4106 - mean_squared_error: 1322.41 - ETA: 0s - loss: 1310.9612 - mean_squared_error: 1310.96 - ETA: 0s - loss: 1322.7247 - mean_squared_error: 1322.72 - ETA: 0s - loss: 1321.3990 - mean_squared_error: 1321.39 - ETA: 0s - loss: 1322.5559 - m

5409/5409 [==============================] - ETA: 1s - loss: 975.9121 - mean_squared_error: 975.91 - ETA: 1s - loss: 869.1976 - mean_squared_error: 869.19 - ETA: 1s - loss: 909.4079 - mean_squared_error: 909.40 - ETA: 1s - loss: 1113.9113 - mean_squared_error: 1113.91 - ETA: 1s - loss: 1080.4508 - mean_squared_error: 1080.45 - ETA: 1s - loss: 1185.8500 - mean_squared_error: 1185.84 - ETA: 0s - loss: 1193.6289 - mean_squared_error: 1193.62 - ETA: 0s - loss: 1198.9958 - mean_squared_error: 1198.99 - ETA: 0s - loss: 1176.0750 - mean_squared_error: 1176.07 - ETA: 0s - loss: 1192.0859 - mean_squared_error: 1192.08 - ETA: 0s - loss: 1195.3663 - mean_squared_error: 1195.36 - ETA: 0s - loss: 1178.8029 - mean_squared_error: 1178.80 - ETA: 0s - loss: 1158.8827 - mean_squared_error: 1158.88 - ETA: 0s - loss: 1156.5416 - mean_squared_error: 1156.54 - ETA: 0s - loss: 1168.8673 - mean_squared_error: 1168.86 - ETA: 0s - loss: 1185.1359 - mean_squared_error: 1185.13 - ETA: 0s - loss: 1199.2854 - mean_

5409/5409 [==============================] - ETA: 2s - loss: 1018.1309 - mean_squared_error: 1018.13 - ETA: 1s - loss: 1220.6711 - mean_squared_error: 1220.67 - ETA: 0s - loss: 1070.9447 - mean_squared_error: 1070.94 - ETA: 0s - loss: 1020.7733 - mean_squared_error: 1020.77 - ETA: 0s - loss: 1069.1827 - mean_squared_error: 1069.18 - ETA: 0s - loss: 1149.3367 - mean_squared_error: 1149.33 - ETA: 0s - loss: 1138.2875 - mean_squared_error: 1138.28 - ETA: 0s - loss: 1183.8222 - mean_squared_error: 1183.82 - ETA: 0s - loss: 1198.3635 - mean_squared_error: 1198.36 - ETA: 0s - loss: 1226.6122 - mean_squared_error: 1226.61 - ETA: 0s - loss: 1232.2262 - mean_squared_error: 1232.22 - ETA: 0s - loss: 1196.2572 - mean_squared_error: 1196.25 - ETA: 0s - loss: 1229.0672 - mean_squared_error: 1229.06 - ETA: 0s - loss: 1259.9836 - mean_squared_error: 1259.98 - ETA: 0s - loss: 1277.7952 - mean_squared_error: 1277.79 - ETA: 0s - loss: 1247.4684 - mean_squared_error: 1247.46 - ETA: 0s - loss: 1259.2494 -

5409/5409 [==============================] - ETA: 1s - loss: 1344.6205 - mean_squared_error: 1344.62 - ETA: 1s - loss: 1071.4253 - mean_squared_error: 1071.42 - ETA: 1s - loss: 1036.2812 - mean_squared_error: 1036.28 - ETA: 1s - loss: 1099.5400 - mean_squared_error: 1099.54 - ETA: 1s - loss: 1099.8691 - mean_squared_error: 1099.86 - ETA: 1s - loss: 1186.1324 - mean_squared_error: 1186.13 - ETA: 1s - loss: 1142.8964 - mean_squared_error: 1142.89 - ETA: 1s - loss: 1154.5969 - mean_squared_error: 1154.59 - ETA: 1s - loss: 1193.5293 - mean_squared_error: 1193.52 - ETA: 0s - loss: 1190.1149 - mean_squared_error: 1190.11 - ETA: 0s - loss: 1204.6704 - mean_squared_error: 1204.67 - ETA: 0s - loss: 1173.4952 - mean_squared_error: 1173.49 - ETA: 0s - loss: 1175.5131 - mean_squared_error: 1175.51 - ETA: 0s - loss: 1204.1102 - mean_squared_error: 1204.11 - ETA: 0s - loss: 1197.5888 - mean_squared_error: 1197.58 - ETA: 0s - loss: 1204.6383 - mean_squared_error: 1204.63 - ETA: 0s - loss: 1164.3188 -

5409/5409 [==============================] - ETA: 0s - loss: 1237.4753 - mean_squared_error: 1237.47 - ETA: 0s - loss: 1224.6197 - mean_squared_error: 1224.61 - ETA: 0s - loss: 1205.2083 - mean_squared_error: 1205.20 - ETA: 0s - loss: 1201.1094 - mean_squared_error: 1201.10 - ETA: 0s - loss: 1238.0129 - mean_squared_error: 1238.01 - ETA: 0s - loss: 1197.5809 - mean_squared_error: 1197.58 - ETA: 0s - loss: 1173.3770 - mean_squared_error: 1173.37 - ETA: 0s - loss: 1181.8547 - mean_squared_error: 1181.85 - ETA: 0s - loss: 1164.7936 - mean_squared_error: 1164.79 - ETA: 0s - loss: 1160.8880 - mean_squared_error: 1160.88 - ETA: 0s - loss: 1188.2101 - mean_squared_error: 1188.21 - ETA: 0s - loss: 1195.6115 - mean_squared_error: 1195.61 - ETA: 0s - loss: 1183.8295 - mean_squared_error: 1183.82 - ETA: 0s - loss: 1156.3611 - mean_squared_error: 1156.36 - ETA: 0s - loss: 1156.3141 - mean_squared_error: 1156.31 - ETA: 0s - loss: 1140.1273 - mean_squared_error: 1140.12 - ETA: 0s - loss: 1142.6125 -

5409/5409 [==============================] - ETA: 1s - loss: 562.6899 - mean_squared_error: 562.68 - ETA: 0s - loss: 1063.2540 - mean_squared_error: 1063.25 - ETA: 0s - loss: 1031.6410 - mean_squared_error: 1031.64 - ETA: 0s - loss: 1115.5414 - mean_squared_error: 1115.54 - ETA: 0s - loss: 1061.3515 - mean_squared_error: 1061.35 - ETA: 0s - loss: 1060.1110 - mean_squared_error: 1060.11 - ETA: 0s - loss: 1097.1570 - mean_squared_error: 1097.15 - ETA: 0s - loss: 1123.3418 - mean_squared_error: 1123.34 - ETA: 0s - loss: 1129.1444 - mean_squared_error: 1129.14 - ETA: 0s - loss: 1148.7500 - mean_squared_error: 1148.74 - ETA: 0s - loss: 1114.4867 - mean_squared_error: 1114.48 - ETA: 0s - loss: 1083.5693 - mean_squared_error: 1083.56 - ETA: 0s - loss: 1092.1099 - mean_squared_error: 1092.10 - ETA: 0s - loss: 1111.9728 - mean_squared_error: 1111.97 - ETA: 0s - loss: 1104.4861 - mean_squared_error: 1104.48 - ETA: 0s - loss: 1108.8731 - mean_squared_error: 1108.87 - ETA: 0s - loss: 1111.3844 - m

5409/5409 [==============================] - ETA: 0s - loss: 1116.5972 - mean_squared_error: 1116.59 - ETA: 1s - loss: 1084.8980 - mean_squared_error: 1084.89 - ETA: 0s - loss: 1193.4125 - mean_squared_error: 1193.41 - ETA: 0s - loss: 1111.9351 - mean_squared_error: 1111.93 - ETA: 0s - loss: 1098.2555 - mean_squared_error: 1098.25 - ETA: 0s - loss: 1071.5796 - mean_squared_error: 1071.57 - ETA: 0s - loss: 1026.6580 - mean_squared_error: 1026.65 - ETA: 0s - loss: 1030.1151 - mean_squared_error: 1030.11 - ETA: 0s - loss: 1046.9573 - mean_squared_error: 1046.95 - ETA: 0s - loss: 1103.2106 - mean_squared_error: 1103.21 - ETA: 0s - loss: 1102.5392 - mean_squared_error: 1102.53 - ETA: 0s - loss: 1106.1721 - mean_squared_error: 1106.17 - ETA: 0s - loss: 1112.9179 - mean_squared_error: 1112.91 - ETA: 0s - loss: 1116.8830 - mean_squared_error: 1116.88 - ETA: 0s - loss: 1117.2788 - mean_squared_error: 1117.27 - ETA: 0s - loss: 1128.4014 - mean_squared_error: 1128.40 - ETA: 0s - loss: 1132.0236 -

5409/5409 [==============================] - ETA: 1s - loss: 943.0657 - mean_squared_error: 943.06 - ETA: 0s - loss: 1080.2628 - mean_squared_error: 1080.26 - ETA: 0s - loss: 1143.9111 - mean_squared_error: 1143.91 - ETA: 0s - loss: 1079.2430 - mean_squared_error: 1079.24 - ETA: 0s - loss: 1103.0441 - mean_squared_error: 1103.04 - ETA: 0s - loss: 1067.2690 - mean_squared_error: 1067.26 - ETA: 0s - loss: 1044.7337 - mean_squared_error: 1044.73 - ETA: 0s - loss: 1061.4271 - mean_squared_error: 1061.42 - ETA: 0s - loss: 1091.1811 - mean_squared_error: 1091.18 - ETA: 0s - loss: 1086.7860 - mean_squared_error: 1086.78 - ETA: 0s - loss: 1103.1055 - mean_squared_error: 1103.10 - ETA: 0s - loss: 1095.6858 - mean_squared_error: 1095.68 - ETA: 0s - loss: 1106.6169 - mean_squared_error: 1106.61 - ETA: 0s - loss: 1117.5541 - mean_squared_error: 1117.55 - ETA: 0s - loss: 1096.5653 - mean_squared_error: 1096.56 - ETA: 0s - loss: 1098.2309 - mean_squared_error: 1098.23 - ETA: 0s - loss: 1092.9593 - m

In [128]:
# See Error, this is much worse than random tree above
y_predi = model.predict_classes(x_test)
score1 = mean_absolute_error(y_test, y_predi)

print("Mean Absolute Error", "", score1)

Mean Absolute Error  62.46003640246479
